# Unegui.mn Apartment Scraper

Collects apartment ads from Unegui.mn 
Unegui.mn is the Mongolia's most widely used advertisement site.

## How it works

1. Opens the website - Uses undetected Chrome to avoid bot detection
2. Collects ad links - Scrolls through search results and collects all apartment ad links
3. Visits each ad - Goes through each ad one by one and extracts data
4. Exports all collected data to one csv file.

## Final collected features
- Floor type
- Balcony
- Garage
- Window type
- Door type
- Number of windows
- Construction progress
- Built year
- Total floor
- Size
- Located floor
- Elevator
- Payment term
- District
- Location
- View count
- Scraped date
- Posted date
- Ad link
- Price
- Number of rooms
- Ad title
- Ad description

Scraped data from 8 districts of Ulaanbaatar:
- Bayanzurkh 
- Sukhbaatar 
- Bayangol 
- Chingeltei 
- Khan Uul 
- Songinokhairkhan 
- Baganuur
- Nalaikh

Combined all district datasets into one final dataset for apartment price prediction modeling. This provides geographic diversity and prevents location bias in the prediction model.

# Bayanzurkh district

In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-bayanzrh/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:      Тагт:   Гараж:  Цонх: Хаалга: Цонхнытоо:      Барилгынявц  \
0  Паркет   1тагттай   Байгаа  Вакум   Төмөр          3  Ашиглалтадорсон   
1  Паркет   2тагттай  Байхгүй  Вакум  Бүргэд          2  Ашиглалтадорсон   
2  Паркет  3+тагттай   Байгаа  Вакум   Төмөр          8  Ашиглалтадорсон   
3  Паркет   1тагттай  Байхгүй  Вакум     Мод          2  Ашиглалтадорсон   
4  Паркет   2тагттай   Байгаа  Вакум  Бүргэд          3  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0                2019              16  74.15м²  ...      УБ   
1                2005               6   49.7м²  ...      УБ   
2                2007              13  356.0м²  ...      УБ   
3                2013               5   32.0м²  ...      УБ   
4                2022              13  76.77м²  ...      УБ   

                       Байршил:       Үзсэн: Scraped_date:  \
0  Баянзүрх, Баянзүрх, Хороо 26   Үзсэн : 48    2025/11/12   
1          Баянзүрх, Зүүн 4 зам  Үзсэн :

# Sukhbaatar district

In [7]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ulan-bator/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:     Тагт:   Гараж:       Цонх: Хаалга: Цонхнытоо:  \
0  Паркет   Тагтгүй  Байхгүй  Төмөрвакум  Бүргэд          8   
1  Паркет  1тагттай  Байхгүй       Вакум  Бүргэд          4   
2  Паркет  1тагттай  Байхгүй       Вакум  Бүргэд          2   
3  Паркет  2тагттай   Байгаа       Вакум  Бүргэд          6   
4  Паркет  1тагттай  Байхгүй       Вакум   Төмөр          3   

         Барилгынявц Ашиглалтандорсонон: Барилгындавхар:   Талбай:  ...  \
0  Ашиглалтадороогүй                2025              10  316.97м²  ...   
1    Ашиглалтадорсон                2007               9    77.2м²  ...   
2    Ашиглалтадорсон                2008               9    40.4м²  ...   
3    Ашиглалтадорсон                2023              16   147.6м²  ...   
4    Ашиглалтадорсон                2025              16    85.0м²  ...   

  Дүүрэг:                        Байршил:       Үзсэн: Scraped_date:  \
0      УБ                 Сүхбаатар, Цирк   Үзсэн : 78    2025/11/12   
1      УБ   Сүхбаатар, Сүх

# Bayangol district

In [8]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-bayangol/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:     Тагт:   Гараж:       Цонх:     Хаалга: Цонхнытоо:  \
0  Паркет  1тагттай  Байхгүй       Вакум      Бүргэд          3   
1  Паркет  1тагттай  Байхгүй       Вакум      Бүргэд          2   
2  Паркет  1тагттай  Байхгүй  Төмөрвакум  Төмөрвакум          7   
3  Паркет   Тагтгүй  Байхгүй       Вакум      Бүргэд          2   
4  Паркет  1тагттай  Байхгүй       Вакум      Бүргэд          2   

       Барилгынявц Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0  Ашиглалтадорсон                2006               6   46.0м²  ...      УБ   
1  Ашиглалтадорсон                2016              12  55.65м²  ...      УБ   
2  Ашиглалтадорсон                2024              16  135.6м²  ...      УБ   
3  Ашиглалтадорсон                2008               9  42.84м²  ...      УБ   
4  Ашиглалтадорсон                2019              16  38.93м²  ...      УБ   

                     Байршил:      Үзсэн: Scraped_date:  \
0      Баянгол, 10-р хороолол   Үзсэн : 5    2025/11/12  

# Chingeltei district

In [9]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-chingeltej/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:     Тагт:   Гараж:  Цонх: Хаалга: Цонхнытоо:      Барилгынявц  \
0  Цемент  2тагттай  Байхгүй  Вакум  Бүргэд          4  Ашиглалтадорсон   
1  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          5  Ашиглалтадорсон   
2  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          5  Ашиглалтадорсон   
3  Паркет  1тагттай  Байхгүй  Вакум   Төмөр          3  Ашиглалтадорсон   
4  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          3  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0                2015               3   23.1м²  ...      УБ   
1                2000               6  100.0м²  ...      УБ   
2                2008              12  122.3м²  ...      УБ   
3                2023               9   68.5м²  ...      УБ   
4                2008              10   67.0м²  ...      УБ   

                         Байршил:      Үзсэн: Scraped_date:  \
0   Чингэлтэй, Чингэлтэй, Хороо 1  Үзсэн : 17    2025/11/12   
1   Чингэлтэй, Чингэлтэй, Хороо 6  Үзсэн : 4

# Khan Uul district

In [10]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-hanuul/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

      Шал:     Тагт:   Гараж:  Цонх: Хаалга: Цонхнытоо:      Барилгынявц  \
0   Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          4  Ашиглалтадорсон   
1   Паркет  1тагттай  Байхгүй  Вакум   Төмөр          5  Ашиглалтадорсон   
2  Ламинат  1тагттай   Байгаа  Вакум   Төмөр          4  Ашиглалтадорсон   
3   Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          3  Ашиглалтадорсон   
4   Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          6  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0                2024              23  109.0м²  ...      УБ   
1                2024              25  79.98м²  ...      УБ   
2                2024              16  104.0м²  ...      УБ   
3                2014              10  69.84м²  ...      УБ   
4                2024              23  213.0м²  ...      УБ   

                     Байршил:       Үзсэн: Scraped_date:  \
0  Хан-Уул, Хан-Уул, Хороо 15  Үзсэн : 167    2025/11/12   
1  Хан-Уул, Хан-Уул, Хороо 18  Үзсэн : 387  

# Songinokhairkhan district

In [11]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-songinohajrhan/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:     Тагт:   Гараж:  Цонх:     Хаалга: Цонхнытоо:      Барилгынявц  \
0  Паркет   Тагтгүй   Байгаа  Вакум      Бүргэд          2  Ашиглалтадорсон   
1  Паркет  1тагттай   Байгаа  Вакум       Төмөр          3  Ашиглалтадорсон   
2  Паркет  1тагттай  Байхгүй  Вакум  Төмөрвакум          3  Ашиглалтадорсон   
3  Паркет  2тагттай  Байхгүй  Вакум      Бүргэд          3  Ашиглалтадорсон   
4  Паркет  1тагттай   Байгаа  Вакум       Төмөр          3  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0                2025               3   57.7м²  ...      УБ   
1                2025              15  78.49м²  ...      УБ   
2                2025              10  72.03м²  ...      УБ   
3                2006               5   86.0м²  ...      УБ   
4                2024              16   51.0м²  ...      УБ   

                                   Байршил:      Үзсэн: Scraped_date:  \
0             Сонгинохайрхан, 21-р хороолол   Үзсэн : 9    2025/11/12   


# Baganuur district

In [12]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-baganuur/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:     Тагт:   Гараж:  Цонх: Хаалга: Цонхнытоо:      Барилгынявц  \
0  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          2  Ашиглалтадорсон   
1  Паркет  1тагттай  Байхгүй  Вакум   Төмөр          3  Ашиглалтадорсон   
2  Паркет  2тагттай  Байхгүй  Вакум  Бүргэд          3  Ашиглалтадорсон   
3  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          2  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар: Талбай:  ... Дүүрэг:  \
0                1980               5  36.0м²  ...      УБ   
1                1986               5  47.0м²  ...      УБ   
2                2024               9  55.3м²  ...      УБ   
3                1985               5  34.0м²  ...      УБ   

                      Байршил:      Үзсэн: Scraped_date:  \
0  Багануур, Багануур, Хороо 1  Үзсэн : 44    2025/11/12   
1  Багануур, Багануур, Хороо 1  Үзсэн : 78    2025/11/12   
2  Багануур, Багануур, Хороо 3  Үзсэн : 55    2025/11/12   
3  Багануур, Багануур, Хороо 1  Үзсэн : 47    2025/11/12   

            

# Nalaikh district

In [13]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

# column names such as floor type, balcony, garage, window type etc
header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

# searches for specific property attribute in the list and extract the value
def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

# extract all unique property ad links from the page
def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

# configuring Chrome options
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# undetected Chrome driver for avoiding bot
driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-nalajh/'   

    # navigate to base url
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # pagination variables
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0

    # loop through pages and collect ad links
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls

        # stop if no new ad on 2 pages
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break

    # converting links
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping each ad
    start_time = time.time()
    successful = 0

    #going through each and collect data
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            # go to each ad
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # parsing page html
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []

            # extracting specific datas
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term

            # get address
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"

            # get view count
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            # scraped date as today
            key_list[16] = date.today().strftime("%Y/%m/%d")

            # posted date
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"

            # ad link
            key_list[18] = listing_url

            # price
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"

            # number of room
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"

            # ad title
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"

            # ad description
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"

            # saving datas to list
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue

    # save the data to csv
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:     Тагт:   Гараж:  Цонх: Хаалга: Цонхнытоо:      Барилгынявц  \
0  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          2  Ашиглалтадорсон   
1  Паркет  2тагттай  Байхгүй  Вакум  Бүргэд          5  Ашиглалтадорсон   
2  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          4  Ашиглалтадорсон   
3  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          4  Ашиглалтадорсон   
4  Паркет  1тагттай  Байхгүй  Вакум  Бүргэд          3  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0                2000               5   32.0м²  ...      УБ   
1                2014               5  77.56м²  ...      УБ   
2                2000               5   68.0м²  ...      УБ   
3                2000               5   55.3м²  ...      УБ   
4                2015               5  45.21м²  ...      УБ   

                  Байршил:       Үзсэн: Scraped_date:  \
0  Налайх, Налайх, Хороо 5   Үзсэн : 80    2025/11/12   
1  Налайх, Налайх, Хороо 7   Үзсэн : 47    2025/11/12   